In [1]:
import polars as pl

In [2]:
import redis

In [3]:
import time

In [4]:
df = pl.scan_parquet("kjoretoyinfo_preppet-kopi.parquet")

In [5]:
df = pl.read_parquet("kjoretoyinfo_preppet-kopi.parquet")

In [6]:
df

tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_aksler_drift,tekn_merke,tekn_modell,tekn_drivstoff,tekn_neste_pkk,tekn_farge,elbil
datetime[μs],datetime[μs],i64,i64,str,str,datetime[μs],str,bool
2021-10-13 00:00:00,2021-10-13 00:00:00,2,3900,"""VITO""","""2""",2025-10-13 00:00:00,"""05""",false
2020-01-29 00:00:00,2020-01-29 00:00:00,2,5750,"""CADDY""","""2""",2026-01-29 00:00:00,"""06""",false
2023-11-29 00:00:00,2023-11-29 00:00:00,1,2600,"""Mustang Mach-E…","""5""",2027-11-29 00:00:00,"""06""",true
2020-01-27 00:00:00,2020-01-27 00:00:00,2,5750,"""CADDY""","""2""",2024-01-27 00:00:00,"""01""",false
2020-05-19 00:00:00,2020-05-19 00:00:00,1,5750,"""CRAFTER""","""2""",2024-05-19 00:00:00,"""01""",false
…,…,…,…,…,…,…,…,…
2021-10-21 00:00:00,2021-10-21 00:00:00,2,3900,"""GLE 350 de 4MA…","""2,5""",2025-10-21 00:00:00,"""03""",false
2023-09-01 00:00:00,2023-09-01 00:00:00,0,320,"""Adora 573 PT""",null,null,"""01""",null
2020-12-29 00:00:00,2020-12-29 00:00:00,1,4000,"""MG ZS EV""","""5""",2024-12-29 00:00:00,"""02""",true


In [7]:
df_filter = df.filter(pl.col("tekn_drivstoff") == "5") 
df_filter = df.filter(pl.col("tekn_reg_f_g_n").dt.strftime("%Y") == "2022")
antall_elbiler = df_filter.shape[0]

In [8]:
antall_elbiler

147086

In [9]:
prosentandel_elbiler = ( antall_elbiler / df.shape[0] ) * 100

prosentandel_elbiler

26.539328316062782

In [10]:
df.columns

['tekn_reg_f_g_n',
 'tekn_reg_eier_dato',
 'tekn_aksler_drift',
 'tekn_merke',
 'tekn_modell',
 'tekn_drivstoff',
 'tekn_neste_pkk',
 'tekn_farge',
 'elbil']

In [11]:
df_9 = df.select("tekn_modell")

In [12]:
#Finn antall gjentakelser for hver bilmodell
modeller = df_filter.groupby("tekn_modell").agg(pl.col("tekn_modell").count().alias("antall_modeller"))

#Skriv ut bilmodeller og antall gjentakelser
for idx, row in enumerate(modeller.iter_rows()):
  print(f"{row[0]}: {row[1]}")  # Tilgang via numeriske indekser


P1515: 2
CBR 1100 XX: 1
Roadmaster 200: 1
Mod Kaf 256A: 1
Mercedes-AMG GT C: 1
2000V712T230: 4
FERREL 112: 5
COUNTRY XCLUSIVE: 1
Sun Living: 1
R5 350: 1
1000V541T209 (R 1000C): 18
Grand Cabrio: 1
E200: 1
Entfellt/N.A/sans objet: 1
850 BOOST RMK Pro 163 Slash: 2
Volvo 744: 1
E 350 CDI: 2
750/135x250: 1
KZ 750 E: 1
JS750-R: 2
AVENTADOR: 1
Corvette Z06 Callaway: 1
CBF1000A: 2
AP3500-DRB: 17
Electric Scooter: 6
K 1200 S: 1
335I XDRIVE: 1
EDELIVER 9: 1
GL1800 Goldwing: 1
M1: 1
MAX: 1
1500M491L220 (SC 4915 inline): 1
XTERRAIN PRO 3900 850 E-TEC: 2
S1945 Inline: 1
S217: 4
FIDDLE: 3
Mitsubishi Eclipse Cross: 108
1000M351T150 (SC 3510): 6
1350M351T150: 5
TGS 33.430: 1
Super Glide: 1
GRAPHITE 3048XLB: 1
GL 350: 1
JUMBO TRAILER: 1
Sonic 700sl: 1
750V431T160 (750RS RIB): 2
525 TDS: 1
Adora 593 UK: 3
HONDA TRX420FE1 RED: 11
COMBO 4X4: 7
CONTURA 766 EF: 1
Bonneville: 1
F 150 RAPTOR: 2
o-3-45: 2
FIAT BARCHETTA: 1
F800GS: 1
CBX 550F2: 1
C20-1121: 1
G781FC: 2
3SGFX: 1
BC6402 CITYCAR  C640: 1
Free: 3
FI

C:\Users\Patri\AppData\Local\Temp\ipykernel_16596\1143300435.py:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  modeller = df_filter.groupby("tekn_modell").agg(pl.col("tekn_modell").count().alias("antall_modeller"))


In [13]:
#Finn største antall og tilhørende modell
max_antall = max(row[1] for row in modeller.iter_rows())
max_modell = next(modell for modell, antall in modeller.iter_rows() if antall == max_antall)

#Skriv ut modellnavn og største antall
print(f"Største antall: {max_antall} for {max_modell}")

Største antall: 8719 for Model Y


In [14]:
# OPPGAVE 12
# Les inn data
df = pl.read_parquet("kjoretoyinfo_preppet-kopi.parquet")

# Sjekk om kolonnen "tekn_reg_f_g_n" eksisterer i datasettet
if "tekn_reg_f_g_n" in df.columns:
    # Ekstraher måneden fra datokolonnen
    registrerings_måned = df["tekn_reg_f_g_n"].dt.month()
    
    # Telle antall registreringer per måned
    måned_counts = registrerings_måned.value_counts()
    
    # Sorter månedsantallene i synkende rekkefølge
    måned_counts = måned_counts.sort("tekn_reg_f_g_n")
    
    # Reverse the sorting order
    måned_counts = måned_counts.reverse()
    
    # Finn måneden med flest registreringer
    mest_populær_måned = måned_counts["tekn_reg_f_g_n"][0]
    
    # Hent ut antall registreringer for den mest populære måneden
    antall_registreringer = måned_counts["count"][0]
    
    print(f"Måneden med flest førstegangsregistreringer er {mest_populær_måned} med {antall_registreringer} registreringer.")
else:
    print("Kolonnen 'tekn_reg_f_g_n' eksisterer ikke i datasettet.")


Måneden med flest førstegangsregistreringer er 12 med 64905 registreringer.
